In [1]:
import sys
import gpytorch
import numpy as np
import pandas as pd
import torch
from datetime import datetime
from scipy.stats import qmc
import itertools
from itertools import combinations_with_replacement, combinations, permutations

import bo_methods_lib
# from bo_methods_lib.bo_methods_lib.bo_functions_generic import gen_theta_set, clean_1D_arrays
from bo_methods_lib.bo_methods_lib.GPBO_Classes_New import * #Fix this later
from bo_methods_lib.bo_methods_lib.GPBO_Class_fxns import * #Fix this later
from bo_methods_lib.bo_methods_lib.analyze_data import * #Fix this later
from bo_methods_lib.bo_methods_lib.GPBO_Classes_plotters import * #Fix this later
import pympler
import pickle

from pympler import asizeof

/afs/crc.nd.edu/user/m/mcarlozo/.conda/envs/Toy_Problem_env/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
#Get all data from ep experiemnts
date_time_str = "2023/09/05/"
save_csv = False
name_cs_str = "CS1"      
meth_name_str_list = ["A1", "B1", "A2", "B2", "C2"]
# study_param_list = ["CONSTANT"]
study_param_list = ["CONSTANT", "DECAY", "BOYLE", "JASRASARIA"]
study_id = "ep"

#Get all ep_data  
df, theta_true = get_study_data_org(date_time_str, name_cs_str, meth_name_str_list, study_id, 
                                    study_param_list, save_csv = False)

In [ ]:
#Get Best Data from ep experiment
save_csv = False
df_best = get_best_data(df, study_id, name_cs_str, theta_true, date_time_str, save_csv)
df_best

In [ ]:
#Get Median Data from ep experiment
save_csv = False
df_median = get_median_data(df, study_id, name_cs_str, theta_true, date_time_str, save_csv)

In [ ]:
#Get Median of Best Data from ep experiment
save_csv = False
df_median_best = get_median_data(df_best, study_id, name_cs_str, theta_true, date_time_str, save_csv)
df_median_best

In [ ]:
#Get all data from sf experiemnts
date_time_str = "2023/09/06/"
# date_time_str = "No_Date/"
save_csv = False
name_cs_str = "CS1" 
meth_name_str_list = ["A1", "B1", "A2", "B2", "C2"]
# meth_name_str_list = ["C2"]
f = [0.5, 0.6, 0.7, 0.8, 0.9, 1.0]
# f = [0.9]
precision = 1
study_param_list = [f"{number:.{precision}f}" for number in f]
study_id = "sf"

#Get all sf_data  
df, theta_true = get_study_data_org(date_time_str, name_cs_str, meth_name_str_list, study_id, 
                                    study_param_list, save_csv = False)

In [ ]:
#Get Best Data from ep experiment
df_best = get_best_data(df, study_id, name_cs_str, theta_true, date_time_str, save_csv)
df_best

In [ ]:
#Get Median Data from ep experiment
df_median = get_median_data(df, study_id, name_cs_str, theta_true, date_time_str, save_csv)

In [ ]:
#Get Median of Best Data from ep experiment
df_median_best = get_median_data(df_best, study_id, name_cs_str, theta_true, date_time_str, save_csv)
df_median_best

In [70]:
# print(df)
file_path = "No_Date/Data_Files/CS1_BO_method_C2_sep_fact_0.9.gz"
with gzip.open(file_path, 'rb') as fileObj:
    results2 = pickle.load(fileObj) 
fileObj.close()

print(len(results2[0].list_gp_emulator_class[0].gp_sim_data.theta_vals))
print(len(results2[0].list_gp_emulator_class[-1].train_data.theta_vals))

# print(results2[0].list_gp_emulator_class[0].gp_sim_data.theta_vals)
# print(results2[0].list_gp_emulator_class[0].train_data.theta_vals)

100
100


In [71]:
# value = df["Exploration Bias"].loc[(df['Run Number'] == 14)]
# pd.set_option('display.float_format', '{:.4e}'.format)
# print(df)
file_path = "2023/09/05/Data_Files/CS2_BO_method_B1_ep_method_CONSTANT.pickle"
with open(file_path, 'rb') as fileObj:
    results2 = pickle.load(fileObj) 
fileObj.close()

file_path = "2023/09/05/Data_Files/CS1_BO_method_B1_ep_method_CONSTANT.pickle"
with open(file_path, 'rb') as fileObj:
    results1 = pickle.load(fileObj) 
fileObj.close()

cs2_tot = 0
cs1_tot = 0

cs2 = 0
cs1 = 0

for i in range(len(results)):
    cs1+= asizeof.asizeof(results[i].configuration)
    cs2+= asizeof.asizeof(results2[i].configuration)

cs2_tot += cs2
cs1_tot += cs1
extra = cs2 - cs1
print(cs2/cs1)
print("from configuration", extra*10**-9, "GB")

cs2 = 0
cs1 = 0

for i in range(len(results)):
    cs1+= asizeof.asizeof(results[i].simulator_class)
    cs2+= asizeof.asizeof(results2[i].simulator_class)

cs2_tot += cs2
cs1_tot += cs1
extra = cs2 - cs1
print(cs2/cs1)
print("from simulator class", extra*10**-9, "GB")

cs2 = 0
cs1 = 0

for i in range(len(results)):
    cs1 += asizeof.asizeof(results[i].exp_data_class)
    cs2 += asizeof.asizeof(results2[i].exp_data_class)
    

cs2_tot += cs2
cs1_tot += cs1
extra = cs2 - cs1
print(cs2/cs1)
print("from exp_data class", extra*10**-9, "GB")

cs2 = 0
cs1 = 0

for i in range(len(results)):
    cs2 += asizeof.asizeof(results2[i].results_df)
    cs1 += asizeof.asizeof(results[i].results_df)
    
cs2_tot += cs2
cs1_tot += cs1
extra = cs2 - cs1
print(cs2/cs1)
print("from pandas df results", extra*10**-9, "GB")

cs2 = 0
cs1 = 0
cs1_iters = 0
cs2_iters = 0
for i in range(len(results)):
    for em in range(len(results[i].list_gp_emulator_class)):
        cs1_iters +=1
        cs1+= asizeof.asizeof(results[i].list_gp_emulator_class[em])
    for em in range(len(results2[i].list_gp_emulator_class)):
        cs2_iters +=1
        cs2 += asizeof.asizeof(results2[i].list_gp_emulator_class[em])

cs2_tot += cs2
cs1_tot += cs1
extra = cs2 - cs1
print(cs2/cs1)
print("from gp emulator class", round(extra*10**-9,2), "GB")

cs2 = 0
cs1 = 0

for i in range(len(results)):
    cs1+= asizeof.asizeof(results[i].heat_map_data_dict)
    cs2+= asizeof.asizeof(results2[i].heat_map_data_dict)

cs2_tot += cs2
cs1_tot += cs1
extra = cs2 - cs1
print(cs2/cs1)
print("from heat map data dictionary",round(extra*10**-9,2), "GB")

print("tot ratio", cs2_tot/cs1_tot)

1.011689070718878
from configuration 4.800000000000001e-07 GB
1.681081081081081
from simulator class 3.0240000000000002e-05 GB
1.706140350877193
from exp_data class 1.932e-05 GB
0.8506998704717336
from pandas df results -0.000142928 GB
93.19797364634054
from gp emulator class 1.45 GB
3640.422748815166
from heat map data dictionary 0.46 GB
tot ratio 113.85050687984153


In [80]:
# print(df)
file_path = "No_Date/Data_Files/CS1_BO_method_C2_sep_fact_0.9.gz"
file_path = "2023/09/05/Data_Files/CS1_BO_method_B1_ep_method_CONSTANT.pickle"
with open(file_path, 'rb') as fileObj:
    results2 = pickle.load(fileObj) 
fileObj.close()

print(len(results2[0].list_gp_emulator_class[0].gp_sim_data.theta_vals))
print(len(results2[0].list_gp_emulator_class[0].gp_val_data.theta_vals))

# print(results2[0].list_gp_emulator_class[0].gp_sim_data.theta_vals)
# print(results2[0].list_gp_emulator_class[0].train_data.theta_vals)

20
400
